In [1]:
import pandas as pd
train = pd.read_csv('train.csv', parse_dates=['date'])
train.head()

,date,store,item,sales
0,2013-01-01,1,1,13
1,2013-01-02,1,1,11
2,2013-01-03,1,1,14
3,2013-01-04,1,1,13
4,2013-01-05,1,1,10


# Feature Engineering

In [2]:
train['dayOfWeek'] = train.date.dt.dayofweek
# train['weekOfYear'] = train.date.dt.weekofyear
# train['dayOfYear'] = train.date.dt.dayofyear
train['month'] = train.date.dt.month
train['year'] = train.date.dt.year - train.date.dt.year.min()
# train['median_store'] = train.groupby(["item", "store"])["sales"].transform("median")
# train["median_store_item_month"] = train.groupby(['month', "item", "store"])["sales"].transform("median")
# train["mean_store_item_week"] = train.groupby(["item", "store",'weekOfYear'])["sales"].transform("mean")

target = train['sales']
train.drop(columns=['date', 'sales'], axis = 1, inplace=True)
train

,store,item,dayOfWeek,month,year
0,1,1,1,1,0
1,1,1,2,1,0
2,1,1,3,1,0
3,1,1,4,1,0
4,1,1,5,1,0
5,1,1,6,1,0
6,1,1,0,1,0
7,1,1,1,1,0
8,1,1,2,1,0
9,1,1,3,1,0


# Build dataset

In [3]:
categorical = list(train.columns)

categorical.remove('year')

cont = ['year']

In [4]:
from sklearn.model_selection import train_test_split
x_train, x_dev, y_train, y_dev = train_test_split(train, target, test_size=.1, random_state=0, shuffle = True)

In [5]:
X_train = []
X_dev = []

X_train.append(y_train.astype('float32').values)
X_dev.append(y_dev.astype('float32').values)

for cat in categorical:
    X_train.append(x_train[cat].values)
    X_dev.append(x_dev[cat].values)

In [6]:
import keras.backend as K

def custom_smape(x, x_):
    return K.mean(2*K.abs(x-x_)/(K.abs(x)+K.abs(x_)))

Using TensorFlow backend.


# Embeddings

In [7]:
cat_sizes = {}
cat_embsizes = {}
for cat in categorical:
    cat_sizes[cat] = train[cat].nunique()
    cat_embsizes[cat] = min(cat_sizes[cat]//2, int(cat_sizes[cat]**0.5))

In [8]:
cat_sizes, cat_embsizes

({'store': 10, 'item': 50, 'dayOfWeek': 7, 'month': 12},
 {'store': 3, 'item': 7, 'dayOfWeek': 2, 'month': 3})

In [9]:
categorical, cont

(['store', 'item', 'dayOfWeek', 'month'], ['year'])

# Model

In [10]:
from keras.layers import Dense, Dropout, Embedding, Input, Reshape, Concatenate
from keras.models import Model
y = Input((len(cont),), name='cont')
y

<tf.Tensor 'cont:0' shape=(?, 1) dtype=float32>

In [11]:
inputs = [y]
concat = [y]
for cat in categorical:
    x = Input((1,), name=cat)
    inputs.append(x)
    
    x = Embedding(cat_sizes[cat]+1, cat_embsizes[cat], input_length=1)(x)
    x = Reshape((cat_embsizes[cat],))(x)
    
    concat.append(x)
    
inputs, concat

([<tf.Tensor 'cont:0' shape=(?, 1) dtype=float32>,
  <tf.Tensor 'store:0' shape=(?, 1) dtype=float32>,
  <tf.Tensor 'item:0' shape=(?, 1) dtype=float32>,
  <tf.Tensor 'dayOfWeek:0' shape=(?, 1) dtype=float32>,
  <tf.Tensor 'month:0' shape=(?, 1) dtype=float32>],
 [<tf.Tensor 'cont:0' shape=(?, 1) dtype=float32>,
  <tf.Tensor 'reshape_1/Reshape:0' shape=(?, 3) dtype=float32>,
  <tf.Tensor 'reshape_2/Reshape:0' shape=(?, 7) dtype=float32>,
  <tf.Tensor 'reshape_3/Reshape:0' shape=(?, 2) dtype=float32>,
  <tf.Tensor 'reshape_4/Reshape:0' shape=(?, 3) dtype=float32>])

In [12]:
y = Concatenate()(concat)
y = Dense(100, activation= 'relu')(y)
y = Dense(1)(y)

In [13]:
model = Model(inputs=inputs, outputs=y)

model.compile(loss=custom_smape, optimizer='adam')

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
store (InputLayer)              (None, 1)            0                                            
__________________________________________________________________________________________________
item (InputLayer)               (None, 1)            0                                            
__________________________________________________________________________________________________
dayOfWeek (InputLayer)          (None, 1)            0                                            
__________________________________________________________________________________________________
month (InputLayer)              (None, 1)            0                                            
__________________________________________________________________________________________________
embedding_

In [14]:
model.fit([train[cont], train.store, train.item, train.dayOfWeek, train.month], 
          target, batch_size=128, epochs=2)

Epoch 1/2
913000/913000 [==============================] - 11s 12us/step - loss: 0.1773
Epoch 2/2
913000/913000 [==============================] - 10s 11us/step - loss: 0.1276


In [15]:
train[cont]

,year
0,0
1,0
2,0
3,0
4,0
5,0
6,0
7,0
8,0
9,0


# Test

In [16]:
train = pd.read_csv('train.csv', parse_dates=['date'])
test = pd.read_csv("test.csv", parse_dates=['date'])

test['year'] = test.date.dt.year-train.date.dt.year.min()
test['month'] = test.date.dt.month
test['day'] = test.date.dt.day
test['dayOfWeek'] = test.date.dt.dayofweek
# test['weekOfYear'] = test.date.dt.weekofyear
# test['dayOfYear'] = test.date.dt.dayofyear

In [17]:
X_test = []
X_test.append(test['year'].astype('float32').values)
for cat in categorical:
    X_test.append(test[cat].values)

In [18]:
test

,id,date,store,item,year,month,day,dayOfWeek
0,0,2018-01-01,1,1,5,1,1,0
1,1,2018-01-02,1,1,5,1,2,1
2,2,2018-01-03,1,1,5,1,3,2
3,3,2018-01-04,1,1,5,1,4,3
4,4,2018-01-05,1,1,5,1,5,4
5,5,2018-01-06,1,1,5,1,6,5
6,6,2018-01-07,1,1,5,1,7,6
7,7,2018-01-08,1,1,5,1,8,0
8,8,2018-01-09,1,1,5,1,9,1
9,9,2018-01-10,1,1,5,1,10,2


In [19]:
preds = model.predict(X_test)
# sample_data = pd.read_csv("sample_submission.csv", index_col=0)
# sample_data['sales'] = preds
# sample_data.to_csv('preds.csv')
preds

array([[10.906397],
       [12.912829],
       [12.835697],
       ...,
       [76.51604 ],
       [81.225746],
       [85.610825]], dtype=float32)